In [2]:
import pandas as pd

from surprise import Dataset, Reader

import joblib

import os

In [3]:
# Tìm đường dẫn thư mục con và thư mục chính
current_dir = os.path.abspath(os.getcwd())
parrent_dir = os.path.dirname(current_dir)
base_dir = os.path.dirname(parrent_dir)

Content-Based

In [4]:
# Tải dữ liệu cho KNN_CBF
meta_data = pd.read_csv(base_dir + "\\Dữ liệu đã được xử lý\\CBF_data.csv")

In [5]:
# Tải model KNN_CBF đã lưu
model_file = parrent_dir + "\\Best recommendation models\\KNN_CBF.pkl"
KNN_CBF = joblib.load(model_file)

In [6]:
def get_content_based_recommendations(itemID, top_n):
  # Truy xuất item features từ mã asin
  itemFeatures = meta_data.loc[meta_data['asin'] == itemID][["encodedCategory", "vectorizedTitle", "encodedBrand"]]

  # Tìm k sản phẩm liên quan đến asin được nhập
  distances, neighbors = KNN_CBF.kneighbors(itemFeatures)

  # Xuất sản phẩm được đề xuất và xóa sản phẩm được nhập
  recommendedItemID = pd.DataFrame(meta_data.iloc[neighbors.flatten()].asin.unique()[:top_n], columns=["asin"])
  recommendedItemID.drop(recommendedItemID[recommendedItemID.asin == itemID].index, inplace=True)

  return recommendedItemID.asin.tolist()

Collabrative-Filtering

In [7]:
# Tải dữ liệu đã được lọc
filtered_ratings = pd.read_csv(base_dir + "\\Dữ liệu đã được xử lý\\CF_data.csv")

reader = Reader(rating_scale=(1, 5))  # Thay đổi rating_scale nếu cần
data = Dataset.load_from_df(filtered_ratings, reader)

In [8]:
# Tải model SVD_CF đã lưu
model_file = parrent_dir + "\\Best recommendation models\\SVD_CF.pkl"
SVD_CF = joblib.load(model_file)

In [9]:
# Tạo trainset
trainset = data.build_full_trainset()

In [10]:
# Hàm để tạo gợi ý
def get_collaborative_filtering_recommendations(user_id, top_n=10):
    # Tạo testset cho người dùng cụ thể
    testset = trainset.build_anti_testset()
    testset = list(filter(lambda x: x[0] == user_id, testset))
    predictions = SVD_CF.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    return [pred.iid for pred in predictions[:top_n]]

Hybrid Recommendtion System

In [12]:
def get_hybrid_recommendations(user_id, product_id, top_n):
    # Lấy danh sách đề xuất từ phương pháp content-based và collaborative filtering
    content_based_recommendations = get_content_based_recommendations(product_id, top_n)
    collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)

    # Tạo danh sách kết quả
    hybrid_recommendations = []

    # Thêm các phần tử trùng nhau vào danh sách kết quả
    for item in content_based_recommendations:
        if item in collaborative_filtering_recommendations:
            hybrid_recommendations.append(item)

    # Đếm số lượng phần tử đã thêm vào danh sách kết quả
    num_added = len(hybrid_recommendations)

    # Nếu danh sách kết quả chưa đủ top_n, thêm xen kẽ từ cả hai nguồn
    while num_added < top_n:

        # Thêm từ collaborative filtering
        if len(collaborative_filtering_recommendations) > 0:
            hybrid_recommendations.append(collaborative_filtering_recommendations.pop(0))
            num_added += 1
            if num_added == top_n:
                break


        # Thêm từ content-based
        if len(content_based_recommendations) > 0:
            hybrid_recommendations.append(content_based_recommendations.pop(0))
            num_added += 1
            if num_added == top_n:
                break

    return hybrid_recommendations

In [13]:
user_id = 'A2GX6DZPHMW9BQ'
product_id = 'B00004UE29'
top_n = 10
recommendations = get_hybrid_recommendations(user_id, product_id, top_n)
# Lọc ra các hàng có mã asin trong mảng
product_RS = meta_data[meta_data['asin'] == product_id]
product_RS.reset_index(inplace=True)
product_RS = product_RS[['category', 'title', 'brand', 'asin']]
print(product_RS)

tada = meta_data[meta_data['asin'].isin(recommendations)].drop_duplicates()
tada.reset_index(inplace=True)
result = tada[['category', 'title', 'brand', 'asin']]

result


    category                           title   brand        asin
0  Recorders  Yamaha YRS24B Soprano Recorder  Yamaha  B00004UE29
1  Recorders  Yamaha YRS24B Soprano Recorder  Yamaha  B00004UE29


,category,title,brand,asin
0,Steel-string Acoustics,"Epiphone DR-100 Acoustic Guitar, Vintage Sunburst",Epiphone,B0002F7IV2
1,Recorders,"Yamaha Alto Horn, Black (YRB302B)",Yamaha,B0002MS7NI
2,Recorders,"Yamaha Alto Horn, Black (YRT304B)",Yamaha,B0002MS7O2
3,Recorders,Yamaha Recorder (YRS-302B),Yamaha,B0002MS7NS
4,Audio Interfaces,Lexicon Multi-Channel Desktop Recording Studio...,Lexicon,B000FFWZKI
5,Vocal,GLS Audio Vocal Microphone ES-58 &amp; Mic Cli...,GLS Audio,B000RKWTPC
6,Microphone Cables,GLS Audio 100ft Mic Cable Patch Cords - XLR Ma...,GLS Audio,B000RO6C1K
7,Solid Body,Saga TC-10 Electric Guitar Kit - T Style,SAGA,B000WEADEW
8,Recorders,Yamaha YRS401 German Ecodear,Yamaha,B00O4MKVGI
9,Recorders,Yamaha YRS402B Baroque ECODEAR,Yamaha,B0134BIH1K
